
# Conditional autoregressive transformer

Train a transformer to predict missing pixel from mnist 

### plan

* note to try padded mnist (relative encoding might require black padding???)
* probably don't need positional encoding?
* create transformer model
* masking 
* randomised masking
* relative position encoding (x - current_x, y - current_y, val)
* train to predict when current pixel missing
* train to predict when 10% are missing
* train to predict when 90% are missing
* train to predict when 99% are missing



In [1]:
import tensorflow as tf
import tensorflow.keras as keras

from PIL import Image, ImageDraw
from IPython.display import display
import numpy as np
import itertools

def display_uint8_image(image):
    if len(image.shape) == 3:
        image = image[:, :, 0]
    if tf.is_tensor(image):
        image = image.numpy()
    display(Image.fromarray(image, "L"))

def display_float32_image(image):
    if len(image.shape) == 3:
        image = image[:, :, 0]
    if tf.is_tensor(image):
        image = image.numpy()
    display_uint8_image(image.astype(np.uint8))





In [2]:
idxs = np.indices([3, 10])
rng = np.random.default_rng()

rng.permutation(idxs[1], axis=1)

array([[7, 4, 8, 3, 5, 0, 9, 6, 1, 2],
       [7, 4, 8, 3, 5, 0, 9, 6, 1, 2],
       [7, 4, 8, 3, 5, 0, 9, 6, 1, 2]])

In [3]:
# reformat MNIST data into (x, y, val) tuples

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data(
    path='mnist.npz'
)

def idxs_to_onehots(idxs):
    onehots = tf.one_hot(idxs, depth=784, dtype=tf.bool, on_value=False, off_value=True)
    return onehots

# takes 2D tensor (batch and index list)
def idxs_to_multihot(idxs):
    onehots = idxs_to_onehots(idxs)
    multihot = tf.math.reduce_all(onehots, axis=len(onehots.shape)-2)
    return multihot

def idxs_to_attention_mask(idxs):
    multihot = idxs_to_multihot(idxs)
    attn_mask = tf.logical_and(multihot[:, :, None], multihot[:, None, :])
    return attn_mask

def mask_to_image_mask(masks):
    image_masks = tf.reshape(masks, [-1, 28, 28])
    return image_masks

def display_mask(mask):
    image_mask = np.array(mask_to_image_mask(mask[np.newaxis, :]), np.uint8)[0]
    image_mask = image_mask * 255
    display_uint8_image(image_mask)

# [row, col, value] tuples
def mnist_to_rcv_tuples(dataset):
    # positional encoding is just indices as floats
    x_idx = np.arange(dataset.shape[1]).astype(np.float32)
    y_idx = np.arange(dataset.shape[2]).astype(np.float32)

    # make 28x28 cartesian product and repeat N times
    idxs = np.transpose([np.repeat(y_idx, len(x_idx)),np.tile(x_idx, len(y_idx))])
    
    all_idxs = np.tile(idxs.reshape(-1), dataset.shape[0]).reshape(-1,784,2)
    # pixel data as float between 0 and 255
    all = dataset.astype(np.float32).reshape(-1, 784, 1)
    
    big = np.dstack((all_idxs, all))
    
    return big

def random_offset_tuples(dataset, dataset_y, min_masked_px=0, max_masked_px=784, random_squares=True):
    tuples = mnist_to_rcv_tuples(dataset)
    offsets = np.random.randint(0, 28, [dataset.shape[0], 2])
    mask_sizes = np.random.randint(min_masked_px, max_masked_px, [dataset.shape[0]])
    tuples[:, :, 0] -= offsets[:, 0, np.newaxis] # row (y) offset
    tuples[:, :, 1] -= offsets[:, 1, np.newaxis] # col (x) offset
    target_idxs = offsets[:, 0] * 28 + offsets[:, 1]
    
    # permutations 
    masks = np.ones([dataset.shape[0], 784], dtype=np.bool_)
    for i in range(dataset.shape[0]):
        # shuffle tuples
        tup_shuffle_idxs = np.random.permutation(784)
        tuples[i] = tuples[i, tup_shuffle_idxs]
        
        idxs = np.random.permutation(784)
                           
        n_mask_px = np.random.randint(min_masked_px, max_masked_px)
        # n mask plus always mask the target idx
        mask_idxs = np.concatenate([idxs[:n_mask_px], target_idxs[i:i+1]])
        
        if random_squares:
            # random square cut out of the masks
            start_row, start_col = np.random.randint(0, 28, [2])
            end_row = np.random.randint(start_row, 28)
            end_col = np.random.randint(start_col, 28)
            height, width = abs(end_row - start_row), abs(end_col - start_col)
            square_row_idxs, square_col_idxs = np.indices([height, width])
            square_row_idxs += start_row
            square_col_idxs += start_col
            square_mask_idxs = square_row_idxs * 28 + square_col_idxs
            square_mask_idxs = square_mask_idxs.reshape(-1)
            
            mask_idxs = np.concatenate([mask_idxs, square_mask_idxs])
            
        masks[i:i+1] = idxs_to_multihot(mask_idxs)
    
    
    
    target_pxs = dataset[np.arange(dataset.shape[0]), offsets[:, 0], offsets[:, 1]].astype(np.float32)
    return {
        "digits": tf.cast(dataset_y, tf.uint8),
        "imgs": tf.cast(dataset, tf.uint8),
        "tuples": tf.cast(tuples, tf.float32),
        "offsets": offsets,
        "target_pxs": target_pxs,
        "target_idxs": target_idxs,
        "masks": tf.cast(masks, tf.bool)
    }

load_saved_dataset = False
if not load_saved_dataset:
    dataset = random_offset_tuples(x_train, y_train)
    test_dataset = random_offset_tuples(x_test, y_test)
    np.savez("dataset.npz", **dataset)
    np.savez("test_dataset.npz", **test_dataset)
else:
    dataset = np.load("dataset.npz")
    test_dataset = np.load("test_dataset.npz")

2021-09-06 17:59:34.883268: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-06 17:59:34.888672: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-06 17:59:34.889026: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-06 17:59:34.889776: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
202

In [4]:
def dataset_summary(d):
    for name, v in d.items():
        print(name, "shape", v.shape)

    for i in range(4):
        print("index", i, "which is a", d["digits"][i])
        display_uint8_image(d["imgs"][i])
        display_mask(d["masks"][i])
        display_uint8_image(tf.constant(d["imgs"][i]) * tf.cast(mask_to_image_mask(d["masks"][np.newaxis, i])[0], tf.uint8))

dataset_summary(dataset)    
dataset_summary(test_dataset)


digits shape (60000,)
imgs shape (60000, 28, 28)
tuples shape (60000, 784, 3)
offsets shape (60000, 2)
target_pxs shape (60000,)
target_idxs shape (60000,)
masks shape (60000, 784)
index 0 which is a tf.Tensor(5, shape=(), dtype=uint8)


index 1 which is a tf.Tensor(0, shape=(), dtype=uint8)


index 2 which is a tf.Tensor(4, shape=(), dtype=uint8)


index 3 which is a tf.Tensor(1, shape=(), dtype=uint8)


digits shape (10000,)
imgs shape (10000, 28, 28)
tuples shape (10000, 784, 3)
offsets shape (10000, 2)
target_pxs shape (10000,)
target_idxs shape (10000,)
masks shape (10000, 784)
index 0 which is a tf.Tensor(7, shape=(), dtype=uint8)


index 1 which is a tf.Tensor(2, shape=(), dtype=uint8)


index 2 which is a tf.Tensor(1, shape=(), dtype=uint8)


index 3 which is a tf.Tensor(0, shape=(), dtype=uint8)


In [5]:

# f_masks = np.array([29, 31, 754, 752])
# print(f_masks)
# f_masks = idxs_to_multihot(f_masks)
# print(f_masks)
# display_mask(f_masks)

# t_masks = np.indices([784])
# print(t_masks)
# t_masks = idxs_to_multihot(t_masks)
# print(t_masks)
# display_mask(t_masks)

In [6]:

print(dataset["tuples"].shape, dataset["offsets"].shape, dataset["target_pxs"].shape)
print(dataset["tuples"][0])
print(dataset["offsets"][0])
print(dataset["target_pxs"][0])

# actual training dataset is 784 x,y,v tuples, but with the spatial location changed so the the pixel to predict is in the center
# and the pixel value to predict removed (via masking)

# then, can mask more and more stuff and compare performance

# we can also use the model in an "iterative" mode to generate images, in an arbitrary order.

(60000, 784, 3) (60000, 2) (60000,)
tf.Tensor(
[[-15. -17.   0.]
 [ -8. -12.   0.]
 [-19. -16.   0.]
 ...
 [ -7.   3.   0.]
 [-24. -11.   0.]
 [-25.   0.   0.]], shape=(784, 3), dtype=float32)
[25 21]
0.0


In [7]:
from tensorflow import keras
from tensorflow.keras import layers

In [8]:
# example from keras docs https://keras.io/examples/nlp/text_classification_with_transformer/
# uses built-in MultiHeadAttention https://keras.io/api/layers/attention_layers/multi_head_attention/

# class TransformerBlock(layers.Layer):
#     def __init__(self, n_embed_dim, n_heads, n_dense_dim, dropout_rate, **kwargs):
#         super(TransformerBlock, self).__init__(**kwargs)
#         self.att = layers.MultiHeadAttention(num_heads=n_heads, key_dim=n_embed_dim)
#         self.dense_net = keras.Sequential([
#             layers.Dense(n_dense_dim, activation='relu'),
#             layers.Dense(n_embed_dim),
#         ])
#         self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
#         self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
#         self.dropout1 = layers.Dropout(dropout_rate)
#         self.dropout2 = layers.Dropout(dropout_rate)
    
#     def call(self, inputs, masks, is_training):
#         # make mask 2D
#         mask = tf.logical_and(masks[:, :, None], masks[:, None, :])
#         attn_output = self.att(inputs, inputs, attention_mask=mask)
#         attn_output = self.dropout1(attn_output, training=is_training)
#         out1 = self.layernorm1(inputs + attn_output)
#         dense_output = self.dense_net(out1)
#         dense_output = self.dropout2(dense_output, training=is_training)
#         return self.layernorm2(out1 + dense_output)

def transformer_block(n_embed_dim, n_heads, n_dense_dim, dropout_rate):
    attn = layers.MultiHeadAttention(num_heads=n_heads, key_dim=n_embed_dim)
    dense_net_1 = layers.Dense(n_dense_dim, activation='relu')
    dense_net_2 = layers.Dense(n_embed_dim)
    layernorm1 = layers.LayerNormalization(epsilon=1e-6)
    layernorm2 = layers.LayerNormalization(epsilon=1e-6)
    dropout1 = layers.Dropout(dropout_rate)
    dropout2 = layers.Dropout(dropout_rate)
    
    def call(inputs, masks, include_residual):
        mask = tf.logical_and(masks[:, :, None], masks[:, None, :])
        attn_output = attn(inputs, inputs, attention_mask=mask)
        attn_output = dropout1(attn_output)
        if include_residual:
            out1 = layernorm1(inputs + attn_output)
        else:
            out1 = layernorm1(attn_output)
        dense_output = dense_net_1(out1)
        dense_output = dense_net_2(dense_output)
        dense_output = dropout2(dense_output)
        return layernorm2(out1 + dense_output)
    
    return call
    

# class TokenAndPositionEmbedding(layers.Layer):
#     def __init__(self, maxlen, vocab_size, embed_dim):
#         super(TokenAndPositionEmbedding, self).__init__()
#         self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
#         self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

#     def call(self, x):
#         maxlen = tf.shape(x)[-1]
#         positions = tf.range(start=0, limit=maxlen, delta=1)
#         positions = self.pos_emb(positions)
#         x = self.token_emb(x)
#         return x + positions



In [9]:
tf.constant([True, False]).dtype

tf.bool

In [10]:
def model(batch_size):

    # no batch size to start makes it simpler
    n_embd = 10

    data_input = keras.Input(shape=[784, 3], batch_size=None)
    mask_input = keras.Input(shape=[784], batch_size=None, dtype=tf.bool)
    m = data_input

    # make n_embd-dimensional input embeddings per pixel from [x, y, v]
    m = layers.Dense(n_embd, activation='relu')(m)
#     m = layers.Dense(20, activation='relu')(m)
#     m = layers.Dense(n_embd, activation=None)(m)
    m = transformer_block(n_embed_dim=n_embd, n_heads=7, n_dense_dim=16, dropout_rate=0.1)(m, masks=mask_input, include_residual=False)
    m = transformer_block(n_embed_dim=n_embd, n_heads=7, n_dense_dim=16, dropout_rate=0.1)(m, masks=mask_input, include_residual=False)
    m = layers.Flatten()(m)
    # m = layers.Dense(10, activation='relu')(m)
    m = layers.Dense(1, activation=None)(m)

    model = keras.Model(inputs=[data_input, mask_input], outputs=[m])

    return model


In [11]:
!nvcc --version
!nvidia-smi

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Wed_Jul_14_19:41:19_PDT_2021
Cuda compilation tools, release 11.4, V11.4.100
Build cuda_11.4.r11.4/compiler.30188945_0
Mon Sep  6 17:59:55 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 470.63.01    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:26:00.0 Off |                  N/A |
|  0%   37C    P2    25W / 170W |   4877MiB /  5926MiB |      0%      Default |
|                               |      

In [12]:
batch_size=16

def get_lr_metric(optimizer):
    def lr(y_true, y_pred):
        return optimizer.lr
    return lr
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
lr_metric = get_lr_metric(optimizer)

txformer = model(batch_size)
txformer.compile(optimizer=optimizer, loss='mse', metrics=['MeanAbsoluteError', lr_metric])
load_saved_model = False
if load_saved_model:
    txformer.load_weights("./models/txformer-var-mask")

txformer.summary()


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 784)]        0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            [(None, 784, 3)]     0                                            
__________________________________________________________________________________________________
tf.__operators__.getitem (Slici (None, 784, 1)       0           input_2[0][0]                    
__________________________________________________________________________________________________
tf.__operators__.getitem_1 (Sli (None, 1, 784)       0           input_2[0][0]                    
______________________________________________________________________________________________

In [13]:
def fit_one_epoch(dataset):
    txformer.fit(x=[dataset["tuples"], dataset["masks"]], y=dataset["target_pxs"], epochs=1, batch_size=batch_size)

# display:
# - before mask
# - mask
# - after mask
# - prediction
def gen_image(dataset, idx):
    img = np.zeros([28, 28])
    mask = dataset["masks"][idx]
    erow, ecol = dataset["offsets"][idx]

    image_mask = np.array(mask_to_image_mask(mask[np.newaxis, :]), np.uint8)[0]
    masked_image = np.copy(dataset["imgs"][idx])
    
    print("MNIST idx", idx, "which is a", y_train[idx])
    
    display_uint8_image(masked_image) # before mask
    display_uint8_image(image_mask * 255) # mask
    masked_image = masked_image * image_mask
    display_uint8_image(masked_image) # after mask
    
    for row in range(28):
        for col in range(28):
            img_tups = np.copy(dataset["tuples"][idx])
            img_tups[:, 0] += erow
            img_tups[:, 1] += ecol
            img_tups[:, 0] -= row
            img_tups[:, 1] -= col
            mask_idx = row*28+col
            targ_idx_mask = idxs_to_onehots(tf.constant([mask_idx]))
            this_mask = tf.logical_and(mask, targ_idx_mask)
            px = txformer([img_tups[np.newaxis, :, :], this_mask])
            img[row, col] = np.clip(px[0, 0], 0, 255)
            
    display_float32_image(img)

gen_image(test_dataset, 0)
gen_image(test_dataset, 1)
gen_image(test_dataset, 2)
gen_image(test_dataset, 3)
gen_image(test_dataset, 4)
gen_image(test_dataset, 5)

MNIST idx 0 which is a 5


MNIST idx 1 which is a 0


MNIST idx 2 which is a 4


MNIST idx 3 which is a 1


MNIST idx 4 which is a 9


MNIST idx 5 which is a 2


In [14]:
fit_one_epoch(dataset)

2021-09-06 18:00:32.732254: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


3750/3750 [==============================] - 223s 59ms/step - loss: 4593.3760 - mean_absolute_error: 42.3356 - lr: 9.9998e-04


In [15]:
gen_image(dataset, 0)
gen_image(dataset, 1)
gen_image(dataset, 2)

MNIST idx 0 which is a 5


MNIST idx 1 which is a 0


MNIST idx 2 which is a 4


In [16]:
fit_one_epoch(dataset)

3750/3750 [==============================] - 223s 59ms/step - loss: 2361.5525 - mean_absolute_error: 26.1150 - lr: 9.9998e-04


In [17]:
gen_image(dataset, 0)
gen_image(dataset, 1)
gen_image(dataset, 2)

MNIST idx 0 which is a 5


MNIST idx 1 which is a 0


MNIST idx 2 which is a 4


In [18]:
fit_one_epoch(dataset)

3750/3750 [==============================] - 223s 59ms/step - loss: 1196.9755 - mean_absolute_error: 17.1338 - lr: 9.9998e-04


In [19]:
gen_image(dataset, 3)
gen_image(dataset, 0)
gen_image(dataset, 5)
gen_image(dataset, 2)
gen_image(dataset, 13)
gen_image(dataset, 10)
gen_image(dataset, 15)
gen_image(dataset, 12)

MNIST idx 3 which is a 1


MNIST idx 0 which is a 5


MNIST idx 5 which is a 2


MNIST idx 2 which is a 4


MNIST idx 13 which is a 6


MNIST idx 10 which is a 3


MNIST idx 15 which is a 7


MNIST idx 12 which is a 3


In [20]:
if not load_saved_model:
    txformer.save_weights("./models/txformer-var-mask")

In [21]:
txformer.save_weights("./models/txformer-square-mask")

In [22]:
# sanity test

gen_image(test_dataset, 0)
gen_image(test_dataset, 1)
gen_image(test_dataset, 2)


MNIST idx 0 which is a 5


MNIST idx 1 which is a 0


MNIST idx 2 which is a 4


In [28]:

full_masks_dataset = random_offset_tuples(x_test, y_test, min_masked_px=763, max_masked_px=764, random_squares=True)

# dataset_summary(full_masks_dataset)

gen_image(full_masks_dataset, 0)
gen_image(full_masks_dataset, 1)
gen_image(full_masks_dataset, 2)

MNIST idx 0 which is a 5


MNIST idx 1 which is a 0


MNIST idx 2 which is a 4
